In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Reshape, Conv2D, MaxPool2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from code.util import *
from code.modelhelper import *

In [24]:
CVECT = 100
UCHARS = 139
NCLASS = 2
BATCH_SIZE = 128
EPOCHS = 300

In [6]:
x = np.load("data/xy/x_char_onehot_45_15.npz")["arr_0"]
y = np.load("data/xy/y_onehot_45_15.npy")
x.shape, y.shape

((60000, 13900), (60000, 2))

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=11
)

In [13]:
model_input = Input((CVECT*UCHARS, ), name="m2_input")
H = Reshape((CVECT, UCHARS, 1), name="m2_rehape")(model_input)

H = Conv2D(
    filters=20,
    kernel_size=(2, 2),
    padding="same",
    activation="tanh",
    name="m2_conv2d_1"
)(H)
H = MaxPool2D((2, 2), name="m2_maxpool2d_1")(H)
H = Dropout(0.5, name="m2_dropout_1")(H)

H = Conv2D(
    filters=20,
    kernel_size=(2, 2),
    padding="same",
    activation="tanh",
    name="m2_conv2d_2"
)(H)
H = MaxPool2D((2, 2), name="m2_maxpool2d_2")(H)
H = Dropout(0.5, name="m2_dropout_2")(H)

H = Conv2D(
    filters=20,
    kernel_size=(2, 2),
    padding="same",
    activation="tanh",
    name="m2_conv2d_3"
)(H)
H = MaxPool2D((2, 2), name="m2_maxpool2d_3")(H)
H = Dropout(0.5, name="m2_dropout_3")(H)

H = Conv2D(
    filters=20,
    kernel_size=(2, 2),
    padding="same",
    activation="tanh",
    name="m2_conv2d_4"
)(H)
H = MaxPool2D((2, 2), name="m2_maxpool2d_4")(H)
H = Dropout(0.5, name="m2_dropout_4")(H)

H = Flatten(name="m2_flatten")(H)
H = Dense(256, activation="tanh", name="m2_dense_1")(H)
H = Dense(64, activation="tanh", name="m2_dense_2")(H)
model_output = Dense(NCLASS, activation="softmax", name="m2_output")(H)

model = Model(model_input, model_output)    
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
m2_input (InputLayer)        [(None, 13900)]           0         
_________________________________________________________________
m2_rehape (Reshape)          (None, 100, 139, 1)       0         
_________________________________________________________________
m2_conv2d_1 (Conv2D)         (None, 100, 139, 20)      100       
_________________________________________________________________
m2_maxpool2d_1 (MaxPooling2D (None, 50, 69, 20)        0         
_________________________________________________________________
m2_dropout_1 (Dropout)       (None, 50, 69, 20)        0         
_________________________________________________________________
m2_conv2d_2 (Conv2D)         (None, 50, 69, 20)        1620      
_________________________________________________________________
m2_maxpool2d_2 (MaxPooling2D (None, 25, 34, 20)        0   

In [25]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

es = EarlyStopping(
    mode="auto",
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(x_test, y_test),
    callbacks=[es]
)

score = model.evaluate(x_test, y_test, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/300
48000/48000 - 32s - loss: 0.3411 - acc: 0.8540 - val_loss: 0.2728 - val_acc: 0.8795
Epoch 2/300


KeyboardInterrupt: 

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred_model = model.predict(x_test)
y_pred = to_bin(y_pred_model)
print("Test recall: {}".format(recall(y_test, y_pred)))

y_test0 = to_1D(y_test)
y_pred0 = to_1D(y_pred)
print(confusion_matrix(y_test0, y_pred0))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# model.save("models/cnn_char.h5")

In [27]:
from tensorflow.keras.backend import tensorflow_

In [ ]:
K.